<a href="https://www.kaggle.com/code/toutatsu/template?scriptVersionId=91006241" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<h1>template notebook</h1>

## import subdirectory from Github?

In [1]:
cd /kaggle/working

/kaggle/working


In [2]:
mkdir -p toutatsu

In [3]:
cd toutatsu

/kaggle/working/toutatsu


In [4]:
!pwd

/kaggle/working/toutatsu


In [5]:
#https://stackoverflow.com/questions/600079/how-do-i-clone-a-subdirectory-only-of-a-git-repository
!git init
!git remote add -f origin "https://github.com/toutatsu/kaggle"
!git config core.sparseCheckout true

!echo "digit-recognizer/*" >> .git/info/sparse-checkout
!echo "template.ipynb" >> .git/info/sparse-checkout

!git pull origin master


Initialized empty Git repository in /kaggle/working/toutatsu/.git/
Updating origin
remote: Enumerating objects: 143, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 143 (delta 51), reused 45 (delta 17), pack-reused 0
Receiving objects: 100% (143/143), 7.23 MiB | 10.96 MiB/s, done.
Resolving deltas: 100% (51/51), done.
From https://github.com/toutatsu/kaggle
 * [new branch]      master     -> origin/master
From https://github.com/toutatsu/kaggle
 * branch            master     -> FETCH_HEAD


In [6]:
ls

digit-recognizer/  template/


In [7]:
cd /kaggle/working/

/kaggle/working


In [8]:
# %%script echo skipping

# import libraries

In [9]:
#defaul libraries
#https://docs.python.org/ja/
import os
import sys
import io

import math
import random
import pprint
import time
import datetime
import typing
import json
import glob
import requests
import warnings
import gc
from pprint import pprint
import re

import numpy as np #https://numpy.org/
import pandas as pd #https://pandas.pydata.org/
import sklearn #https://scikit-learn.org/stable/

import matplotlib.pyplot as plt #https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.html
%matplotlib inline

import seaborn as sns
sns.set()

from tqdm import tqdm #https://tqdm.github.io/

import torch #https://pytorch.org/
import transformers #https://huggingface.co/transformers/

import torchvision
import torchtext

In [10]:
print(os.curdir)
for dirname, _, filenames in os.walk('.'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

.
./__notebook__.ipynb
./toutatsu/digit-recognizer/README.md
./toutatsu/digit-recognizer/MNIST_Pytorch_CNN.ipynb
./toutatsu/digit-recognizer/MNIST_EDA.ipynb
./toutatsu/digit-recognizer/MNIST_Pytorch_CNN_.ipynb
./toutatsu/digit-recognizer/misc/svm.py
./toutatsu/digit-recognizer/misc/mnist.py
./toutatsu/digit-recognizer/misc/PCA.py
./toutatsu/digit-recognizer/misc/knn.py
./toutatsu/.git/index
./toutatsu/.git/FETCH_HEAD
./toutatsu/.git/description
./toutatsu/.git/HEAD
./toutatsu/.git/config
./toutatsu/.git/info/sparse-checkout
./toutatsu/.git/info/exclude
./toutatsu/.git/logs/HEAD
./toutatsu/.git/logs/refs/heads/master
./toutatsu/.git/logs/refs/remotes/origin/master
./toutatsu/.git/objects/pack/pack-64f43e2d2fbb5fbd5c2841856827e01c6ea20c55.idx
./toutatsu/.git/objects/pack/pack-64f43e2d2fbb5fbd5c2841856827e01c6ea20c55.pack
./toutatsu/.git/hooks/pre-push.sample
./toutatsu/.git/hooks/commit-msg.sample
./toutatsu/.git/hooks/pre-applypatch.sample
./toutatsu/.git/hooks/update.sample
./toutatsu/

# Configuration

In [11]:
class CFG():
    
    data_path="./"#../input/tabular-playground-series-mar-2021/"
    debug=True
    seed=0

    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    batch_size=1#32
    epochs=30
    learning_rate=0.1
    kFold=5

    amp=False


    #高速化関連
    #https://qiita.com/sugulu_Ogawa_ISID/items/62f5f7adee083d96a587

    #GPU 遅くなるらしい↓
    torch.backends.cudnn.deterministic = True

    #イテレーションごとのnnの順伝搬および誤差関数の 計算手法がある程度一定であれば、torch.backends.cudnn.benchmark = Trueで GPU での計算が高速化
    torch.backends.cudnn.benchmark = False


def set_seed(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    #tf.random.set_seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(CFG.seed)

print(CFG.device)
if torch.cuda.is_available():
    torch.cuda.set_device(CFG.device)
    print(torch.cuda.current_device())
    print(torch.cuda.get_device_name())

if CFG.debug:
    CFG.epochs=5

cpu


# Color

In [12]:
def color(string,fg='DEFAULT',bg='DEFAULT',fg_rgb=None,bg_rgb=None,style='END'):
    colors=['BLACK','RED','GREEN','YELLOW','BLUE','PURPLE','CYAN','WHITE','8','DEFAULT']
    styles=['END','BOLD','2','3','UNDERLINE','5','6','REVERSE','INVISIBLE','9']

    fg=f'\033[3{colors.index(fg)}m'
    bg=f'\033[4{colors.index(bg)}m'
    style=f'\033[0{styles.index(style)}m'

    if fg_rgb:fg=f"\033[38;2;{fg_rgb[0]};{fg_rgb[1]};{fg_rgb[2]}m"
    if bg_rgb:bg=f"\033[48;2;{bg_rgb[0]};{bg_rgb[1]};{bg_rgb[2]}m"

    return style+fg+bg+str(string)+'\033[0m'

In [13]:
print(color("color test","GREEN",style='BOLD'))
print(color("color test","RED","BLUE",style='UNDERLINE'))
print(color("color test","CYAN","WHITE",style='INVISIBLE'))
print(color("color test","CYAN",style='REVERSE'))
print(color("aaa",fg_rgb=(150,150,255)))
# print(color("bbb",fg_rgb=(0,250,50),bg_rgb=(0,0,255)))
# print(color("ccc"))
# print(color('style',bg="RED",fg='BLUE'))
# print('\033[07m'+"Aa")


color test
color test
color test
color test
aaa


In [14]:
def what_is(obj):
    print("type:",type(obj))
    print("-"*40+"help"+"-"*40)
    print(help(obj))
    print("-"*40+"dir"+"-"*40)
    print(dir(obj))

In [15]:
target_columns=['target']
def feature_columns(df):
    for col in df.columns:
        if col in target_columns:df=df.drop(columns=col)
    return  df.columns
def numerical_columns(df):
    return df[feature_columns(df)].select_dtypes(include=['int64','float64']).columns
def categorical_columns(df):
    return df[feature_columns(df)].select_dtypes(exclude=['int64','float64']).columns

read data

In [16]:
%%script echo skipping

train=pd.read_csv(os.path.join(CFG.data_path,"train.csv"),index_col='id')
test=pd.read_csv(os.path.join(CFG.data_path,"test.csv"),index_col='id')
sample_submission=pd.read_csv(os.path.join(CFG.data_path,"sample_submission.csv"),index_col='id')

if CFG.debug:
    #train=train[:len(train)//50]
    train=train.sample(frac=0.1,random_state=CFG.seed)

df=pd.concat([train,test])
# train=df[:len(train)].copy()
# test=df[len(train):].copy()

data={
    'train':pd.DataFrame(columns=['feature','target']),
    'val':pd.DataFrame({'feature':val.feature,'target':val['target']}),
    'test':pd.DataFrame(test),
}



skipping


In [17]:
class datapath:
    def __init__(self,path):
        self.data_path=path
    def __getitem__(self,idx):
        return self.data_path+str(idx)+'.png'
    
data={
    'train':datapath('/kaggle/working/train/'),
    'test':datapath('/kaggle/working/test/'),
}

submission

In [18]:
import pandas as pd

In [19]:
submission=pd.DataFrame(index=pd.Series(name='row_id',dtype='int'),columns=['idx','target'])


submission.idx=range(5)
submission.target=[1,0,0,1,1]

#submission.sort_index(inplace=True)

#submission.to_csv("submission.csv",index=idx)

submission.head()

,idx,target
row_id,,
0,0,1
1,1,0
2,2,0
3,3,1
4,4,1
